connecting multiple files for exception

load_missing_file --> retry_file_load --> data analysis

In [ ]:
from pyspark.sql.functions import col
import time

#Number of retries
max_retries = 3
retries = 0
success = False

# Define the transformation logic
def transform_data():
  df = spark.read.format("csv").option("header", "true").load("/dbfs/mnt/data/input_data.csv")
  df_transformed = df.withColumn("Salary", col("Salary").cast("double") * 1.1)
  df_transformed.write.format("delta").mode("overwrite").save("/delta/transformed_data")
  print("Data transformed successfully.")

# Retry loop
while retries < max_retries and not success:
  try:
    transform_data()
    success = True
  except Exception as e:
    retries += 1
    print(f"Transformation failed. Retry (retries}/{max_retries}")
    time.sleep(5)
if not success:
  print("Transformation failed after max retries.")